In [1]:
### same as main.py but as a jpynb

import open3d as o3d 
import numpy as np
import trimesh

import datato3D as d3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Bounding Boxes

In [2]:

path = 'testfiles\Muensterhuegel.txt'
bbox_product = d3d.read_lv95_do_rectangular_bbox(path)
#
print(bbox_product)

POLYGON ((2611417.8 1267159.9, 2611695.8 1267159.9, 2611695.8 1267471.4, 2611417.8 1267471.4, 2611417.8 1267159.9))


In [3]:
bbox_working_region = d3d.buffer_polygon(bbox_product, 0.1)
#
print(bbox_working_region)

POLYGON ((2611299.9 1267159.9, 2611299.9 1267471.4, 2611300.467720725 1267482.9562208448, 2611302.16541544 1267494.4011489656, 2611304.976734417 1267505.6245634481, 2611308.874603117 1267516.5183766757, 2611313.821482933 1267526.9776752717, 2611319.7697327095 1267536.901730473, 2611326.6620675484 1267546.1949682028, 2611334.4321104977 1267554.7678895018, 2611343.005031797 1267562.5379324513, 2611352.298269527 1267569.4302672904, 2611362.222324728 1267575.3785170666, 2611372.681623324 1267580.325396883, 2611383.5754365516 1267584.2232655827, 2611394.798851034 1267587.0345845595, 2611406.243779155 1267588.7322792746, 2611417.8 1267589.2999999998, 2611695.8 1267589.2999999998, 2611707.3562208447 1267588.7322792746, 2611718.8011489655 1267587.0345845595, 2611730.024563448 1267584.2232655827, 2611740.918376676 1267580.325396883, 2611751.3776752716 1267575.3785170666, 2611761.3017304726 1267569.4302672904, 2611770.5949682025 1267562.5379324513, 2611779.167889502 1267554.7678895018, 2611786.9

# DEM


In [4]:
### if your DEM-file does not cover the area, try d3d.download_raster_files() and stitch_raster_files() to get a DEM that covers the area
### automatically download and stich raster files from the csv you get when you draw a polygon on https://www.swisstopo.admin.ch/de/hoehenmodell-swissalti3d

# csv_path = "example_Belinzona/ch.swisstopo.swissalti3d-ix4nQqFB.csv"
# output_directory = "example_Belinzona/"
# raster_files = d3d.download_raster_files(csv_path, output_directory)

# output_filename = "example_Belinzona/merged_raster.tif"
# d3d.stitch_raster_files(raster_files, output_filename)

In [23]:

dem_path = 'testfiles\swissalti3d_2019_2611-1267_0.5_2056_5728.tif'
out_image, out_transform, src = d3d.read_raster_dem_cut_to_bbox(dem_path, bbox_working_region)
out_image = d3d.slice_out_image(out_image)
#
print(out_image.shape)

(1095, 1029)


In [25]:
# The parameters are (out_image, out_transform, resolution, max_triangle_size, max_triangle_height)
mesh, pcd = d3d.dem_to_mesh(out_image, out_transform, 0.1, 30, 7)

# Visualize the mesh and pointcloud
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)


In [26]:
mesh_cropped = d3d.cutting_mesh_with_bbox(mesh, bbox_product, 100, 300)
# Visualize the cut mesh and pointcloud
print(mesh, mesh_cropped)

o3d.visualization.draw_geometries([mesh_cropped], mesh_show_wireframe=True, mesh_show_back_face=True)


TriangleMesh with 19000 points and 37803 triangles. TriangleMesh with 4537 points and 8805 triangles.


In [27]:
# afterwork on the mesh
mesh_smooth = mesh_cropped.filter_smooth_taubin(number_of_iterations=3)
mesh_smooth.compute_triangle_normals()

TriangleMesh with 4537 points and 8805 triangles.

In [18]:
# writing the surface mesh to a file
output_file = 'testfiles/mesh_surface.ply'
o3d.io.write_triangle_mesh(output_file, mesh_smooth)


True

In [29]:
thickness = 40 #from lowest point of DEM to bottom of the output stl, standard value is calculated to be at sea level
# create a solid mesh from the surface mesh
solid_mesh = d3d.surface_to_volume(mesh_smooth, thickness)

# Visualize the updated mesh
o3d.visualization.draw_geometries([solid_mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

# Export the extruded mesh to a file
output_file = 'testfiles/mesh_solid.stl'
o3d.io.write_triangle_mesh(output_file, solid_mesh) 



[Open3D WARNING] Ignoring attribute 'normals' for TensorMap with primary key 'positions'
[Open3D WARNING] Ignoring attribute 'colors' for TensorMap with primary key 'positions'
[Open3D WARNING] Ignoring attribute 'normals' for TensorMap with primary key 'indices'


True

# Buildings


In [30]:
### shrinking bbox so the buildings will fit proparly, but can result on slicing failures
shrink_factor = 0.001
bbox_product = d3d.shrink_bbox(bbox_product, shrink_factor)

In [32]:
### change to trimesh as it can cut through the middle of faces which open3d cannot
mesh_path = 'testfiles/3DStadtmodell_beschränkt.obj'
mesh = trimesh.load_mesh(mesh_path)

mesh_cut = d3d.cutting_mesh_through_middle_of_faces(mesh, bbox_product)
mesh_cut.show()


In [22]:
# Define edge_vertices
edge_vertices = mesh_cut.outline().vertices

### Filtering the vertices so that only edge points are left for each side
coordinates = np.array(edge_vertices[:, :3])

combined_mesh = mesh_cut

combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'east')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'west')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'north')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'south')

### trying to get more slice reliability
trimesh.repair.fill_holes(combined_mesh)

output_file = 'testfiles/stadtmodell_bbox.stl'
combined_mesh.export(output_file)

combined_mesh.show()

# testing
